In [2]:
import numpy as np
from scipy.linalg import expm
from scipy.optimize import minimize

In [3]:
def pauli_to_matrix(pauli_word: str) -> np.ndarray:
    """
    Convert a Pauli string into its matrix representation using Kronecker products.
    For example, 'XZ' -> kron(X, Z).
    """
    # Define Pauli matrices
    I = np.array([[1, 0], [0, 1]], dtype=complex)
    X = np.array([[0, 1], [1, 0]], dtype=complex)
    Y = np.array([[0, -1j], [1j, 0]], dtype=complex)
    Z = np.array([[1, 0], [0, -1]], dtype=complex)
    pauli_map = {'I': I, 'X': X, 'Y': Y, 'Z': Z}
    
    # Start with the matrix for the first Pauli and Kronecker product sequentially
    if len(pauli_word) == 0:
        raise ValueError("Pauli word must be a non-empty string.")
    result = pauli_map.get(pauli_word[0])
    if result is None:
        raise ValueError(f"Invalid Pauli character: {pauli_word[0]}")
    for p in pauli_word[1:]:
        if p not in pauli_map:
            raise ValueError(f"Invalid Pauli character: {p}")
        result = np.kron(result, pauli_map[p])
    return result

def optimize_pauli_word(pauli_word: str, 
                        lambda0: float = 1.0, 
                        beta0: float = 0.1, 
                        theta0: float = 0.1, 
                        phi0: float = 0.0,
                        method: str = 'BFGS', 
                        callback=None):
    """
    Optimize the parameters (lambda, beta, theta, phi) to approximate the given Pauli word 
    using the ansatz from Eq. (15). Returns the SciPy optimization result.
    
    Parameters:
        pauli_word (str): Pauli string for the target operator.
        lambda0, beta0, theta0, phi0: Initial guesses for λ, β, θ, φ.
        method (str): Optimization method for scipy.optimize.minimize (default 'BFGS').
        callback (callable): Optional function called after each iteration with current params.
    """
    # Convert Pauli word to target matrix
    W_target = pauli_to_matrix(pauli_word)
    L = W_target.shape[0]  # Dimension of the target operator (2^N)
    
    # Precompute ladder operators for the bosonic mode of dimension L
    # b (annihilation) and b_dag (creation) in the Fock basis (0,1,...,L-1)
    b = np.zeros((L, L), dtype=complex)
    for n in range(L-1):
        b[n, n+1] = np.sqrt(n+1)
    b_dag = b.T.conj()
    # Define the matrix for (b_dag - b), used in computing D(β)
    B = b_dag - b  # This is anti-Hermitian; expm(beta * B) gives the displacement operator for real beta.
    
    # Define the objective function (loss) to minimize
    def loss(x):
        # x is [lambda, beta, theta, phi]
        lam, beta, theta, phi = x
        
        # Compute single-qubit rotation R(θ, φ) on the ancilla (2x2 matrix)
        # R = cos(θ/2)*I - i * sin(θ/2) * (cos φ * X + sin φ * Y)
        cos_half = np.cos(theta/2.0)
        sin_half = np.sin(theta/2.0)
        cosφ = np.cos(phi)
        sinφ = np.sin(phi)
        # Pauli X and Y matrices for qubit (2x2)
        X_qubit = np.array([[0, 1], [1, 0]], dtype=complex)
        Y_qubit = np.array([[0, -1j], [1j, 0]], dtype=complex)
        R = cos_half * np.eye(2, dtype=complex) - 1j * sin_half * (cosφ * X_qubit + sinφ * Y_qubit)
        
        # Displacement operators on the mode: D_plus = D(β/2), D_minus = D(-β/2)
        # Use expm for the matrix exponential. For real β, D(-β/2) is the Hermitian conjugate of D(β/2).
        D_plus = expm((beta/2.0) * B)            # LxL matrix
        D_minus = D_plus.conj().T               # use conjugate transpose for D(-β/2)
        
        # Construct the ECD(β) gate as a 2L x 2L matrix:
        # ECD = |1><0| ⊗ D_plus + |0><1| ⊗ D_minus.
        # We build it in block form for efficiency.
        zeros_L = np.zeros((L, L), dtype=complex)
        top = np.hstack([zeros_L, D_minus])   # [0, D(-β/2)]
        bottom = np.hstack([D_plus, zeros_L]) # [D(β/2), 0]
        ECD_matrix = np.vstack([top, bottom])  # combine top and bottom blocks
        
        # Apply rotation on qubit (ancilla) before ECD. We do Kron(R, I_L) to apply R on ancilla.
        R_extended = np.kron(R, np.eye(L, dtype=complex))  # 2L x 2L matrix
        U_ER = ECD_matrix @ R_extended  # Combined unitary for this ansatz block
        
        # Scale by lambda
        V_matrix = lam * U_ER
        
        # Extract the block where ancilla is |0> (top-left LxL block)
        V_block00 = V_matrix[0:L, 0:L]
        
        # Compute Frobenius norm squared of the difference between V_block00 and W_target
        diff = V_block00 - W_target
        cost = np.linalg.norm(diff, 'fro')**2
        return cost.real  # cost should be real; take real part for safety
    
    # Initial parameter vector
    x0 = np.array([lambda0, beta0, theta0, phi0], dtype=float)
    
    # Run the SciPy optimization
    result = minimize(loss, x0, method=method, callback=callback)
    return result

# Example usage:
result = optimize_pauli_word("XZ", lambda0=1.0, beta0=0.1, theta0=0.1, phi0=0.0)
print("Final cost:", result.fun)
print("Optimized parameters:", result.x)
print("Optimization result:", result)


Final cost: 4.00000000000111
Optimized parameters: [ 9.89991533e-01  1.00000000e-01 -9.08821257e-07 -1.29409801e-04]
Optimization result:   message: Optimization terminated successfully.
  success: True
   status: 0
      fun: 4.00000000000111
        x: [ 9.900e-01  1.000e-01 -9.088e-07 -1.294e-04]
      nit: 1
      jac: [ 0.000e+00  0.000e+00 -2.027e-06  0.000e+00]
 hess_inv: [[1 0 0 0]
            [0 1 0 0]
            [0 0 1 0]
            [0 0 0 1]]
     nfev: 15
     njev: 3


In [ ]:
import numpy as np
from scipy.linalg import expm
from scipy.optimize import minimize

def pauli_to_matrix(pauli_word: str) -> np.ndarray:
    """
    Convert a Pauli string into its matrix representation via Kronecker products.
    
    Parameters:
        pauli_word (str): A string like 'XIZY' where each character is I, X, Y, or Z.
        
    Returns:
        np.ndarray: The resulting 2^N x 2^N complex matrix.
    """
    # Define basic Pauli matrices
    I = np.array([[1, 0], [0, 1]], dtype=complex)
    X = np.array([[0, 1], [1, 0]], dtype=complex)
    Y = np.array([[0, -1j], [1j, 0]], dtype=complex)
    Z = np.array([[1, 0], [0, -1]], dtype=complex)
    pauli_map = {'I': I, 'X': X, 'Y': Y, 'Z': Z}
    
    if not pauli_word:
        raise ValueError("Pauli word must be a non-empty string.")
    
    result = pauli_map.get(pauli_word[0])
    if result is None:
        raise ValueError(f"Invalid Pauli character: {pauli_word[0]}")
    for char in pauli_word[1:]:
        if char not in pauli_map:
            raise ValueError(f"Invalid Pauli character: {char}")
        result = np.kron(result, pauli_map[char])
    return result

def rotation_gate(theta: float, phi: float) -> np.ndarray:
    """
    Constructs the single-qubit rotation R(theta, phi) defined by:
      R(theta, phi) = exp( - i (theta/2) (cos(phi) X + sin(phi) Y) )
    
    Parameters:
        theta (float): Rotation angle.
        phi (float): Phase angle.
        
    Returns:
        np.ndarray: A 2x2 rotation matrix.
    """
    X = np.array([[0, 1], [1, 0]], dtype=complex)
    Y = np.array([[0, -1j], [1j, 0]], dtype=complex)
    R = expm( - 1j * (theta/2) * (np.cos(phi) * X + np.sin(phi) * Y) )
    return R

def get_bosonic_operators(L: int):
    """
    Generates the annihilation operator 'b', its Hermitian conjugate 'b_dag',
    and the operator B = b_dag - b in the Fock basis of dimension L.
    
    Parameters:
        L (int): Fock space cutoff dimension.
        
    Returns:
        tuple: (b, b_dag, B) where each is an L x L numpy array.
    """
    b = np.zeros((L, L), dtype=complex)
    for n in range(L - 1):
        b[n, n+1] = np.sqrt(n + 1)
    b_dag = b.T.conj()
    B = b_dag - b  # This operator is used to construct the displacement operator for real beta.
    return b, b_dag, B

def displacement_operator(beta: float, b: np.ndarray) -> np.ndarray:
    """
    Constructs the displacement operator D(alpha) = exp(beta bconj- beta^* b).
    
    Parameters:
        beta (float): Displacement amplitude (can be beta/2).
        b (np.ndarray): The bosonic annihilation operator.
        
    Returns:
        np.ndarray: The displacement operator (L x L matrix).
    """
    return expm(beta * b.T.conj() - beta.conjugate() * b)

def ECD_gate(beta: float, L: int, b: np.ndarray) -> np.ndarray:
    """
    Constructs the Echoed Conditional Displacement (ECD) gate:
      ECD(beta) = |1><0| ⊗ D(beta/2) + |0><1| ⊗ D(-beta/2)
    
    Parameters:
        beta (float): Displacement parameter.
        L (int): Dimension of the bosonic mode (Fock cutoff).
        b (np.ndarray): The bosonic annihilation operator.
        
    Returns:
        np.ndarray: The full 2L x 2L ECD gate.
    """
    # Compute the displacement operators.
    D_plus = displacement_operator(beta/2, b)  # D(beta/2)
    D_minus = displacement_operator(-beta/2, b) # D(-beta/2)
    
    zeros_L = np.zeros((L, L), dtype=complex)
    # Construct block matrix: top block corresponds to ancilla |0>, bottom to |1>
    top = np.hstack([zeros_L, D_minus])   # |0><1| block
    bottom = np.hstack([D_plus, zeros_L])   # |1><0| block
    ECD = np.vstack([top, bottom])
    return ECD

def ansatz_gate(lam: float, beta: float, theta: float, phi: float, L: int, b: np.ndarray) -> np.ndarray:
    """
    Constructs the parameterized ansatz gate V(λ, β, θ, φ):
      V = λ * [ECD(beta) * (R(theta, phi) ⊗ I_L)]
      
    where R(theta, phi) is the single-qubit rotation and ECD(beta) is the echoed conditional
    displacement gate.
    
    Parameters:
        lam (float): Scaling coefficient.
        beta (float): Displacement parameter for ECD.
        theta (float): Rotation angle for R.
        phi (float): Phase angle for R.
        L (int): Fock cutoff dimension.
        b (np.ndarray): The bosonic annihilation operator.
        
    Returns:
        np.ndarray: The 2L x 2L ansatz gate.
    """
    # Single-qubit rotation on the ancilla.
    R = rotation_gate(theta, phi)
    R_extended = np.kron(R, np.eye(L, dtype=complex))  # Extend to ancilla ⊗ bosonic mode.
    # ECD gate.
    ECD = ECD_gate(beta, L, b)
    # Combined unitary.
    U_ER = ECD @ R_extended
    return lam * U_ER

def extract_ancilla_block(V: np.ndarray, L: int) -> np.ndarray:
    """
    Extracts the block of the full ansatz gate corresponding to the ancilla being in |0>.
    
    For a 2L x 2L matrix V, returns the top-left L x L block.
    
    Parameters:
        V (np.ndarray): The full 2L x 2L unitary matrix.
        L (int): Dimension of the bosonic mode.
        
    Returns:
        np.ndarray: The L x L matrix corresponding to the ancilla |0> subspace.
    """
    return V[:L, :L]

def loss_function(x: np.ndarray, W_target: np.ndarray, L: int, b: np.ndarray) -> float:
    """
    Computes the loss function as defined in Eq. (15):
      F = (1 / L^2) * sum_{n,m} | <0,n|(I ⊗ W_target)|0,m> - <0,n|V|0,m> |^2,
      
    where V is the ansatz gate built from parameters x = [λ, β, θ, φ].
    
    Parameters:
        x (np.ndarray): Parameter vector [lam, beta, theta, phi].
        W_target (np.ndarray): Target operator matrix from the given Pauli word.
        L (int): Dimension of the bosonic mode.
        B (np.ndarray): The bosonic annihilation operator.
        
    Returns:
        float: The loss value (Frobenius norm squared difference, normalized by L^2).
    """
    lam, beta, theta, phi = x
    V = ansatz_gate(lam, beta, theta, phi, L, b)
    V_block = extract_ancilla_block(V, L)
    diff = V_block - W_target
    cost = 0.0
    for n in range(L):
        for m in range(L):
            cost += np.abs(diff[n, m])**2
    cost /= (L**2)
    return cost.real

def optimize_pauli_word(pauli_word: str, 
                        lambda0: float = 1.0, 
                        beta0: float = 0.1, 
                        theta0: float = 0.1, 
                        phi0: float = 0.0,
                        method: str = 'BFGS', 
                        callback=None):
    """
    Optimizes the parameters (λ, β, θ, φ) to approximate the target operator defined by the 
    given Pauli word via the ansatz gate V.
    
    Parameters:
        pauli_word (str): The Pauli string for the target operator.
        lambda0, beta0, theta0, phi0: Initial guesses for the parameters.
        method (str): Optimizer method to use (default 'BFGS').
        callback (callable): Optional callback called with the current parameter vector.
        
    Returns:
        OptimizeResult: The result object from scipy.optimize.minimize.
    """
    # Convert the Pauli word to its matrix representation.
    W_target = pauli_to_matrix(pauli_word)
    # W_target = expm(-1j*W_target)
    L = W_target.shape[0]  # Fock space cutoff is 2^N where N is the length of the Pauli word.
    
    # Generate bosonic operators.
    bvec, bdaggervec, B = get_bosonic_operators(L)
    
    # Define the objective function (closure) for optimization.
    def objective(x):
        return loss_function(x, W_target, L, bvec)
    
    # Initial parameter vector.
    x0 = np.array([lambda0, beta0, theta0, phi0], dtype=float)
    
    # Run the optimizer.
    result = minimize(objective, x0, method=method, callback=callback)
    return result


# Test pauli_to_matrix
# pauli_str = "ZIII"
# mat = pauli_to_matrix(pauli_str)
# print("Pauli matrix for", pauli_str, ":\n", mat)

# Test rotation_gate
# R = rotation_gate(0.5, 1.0)
# print("Rotation gate R(0.5, 1.0):\n", R)

# # Test bosonic operators and displacement
# L = 16  # For example, for 2 qubits we have L = 2^2 = 4.
# _, _, B = get_bosonic_operators(L)
# D = displacement_operator(0.1, B)
# print("Displacement operator D(0.1):\n", D)

# # Test ECD gate
# ECD = ECD_gate(0.2, L, B)
# print("ECD gate for beta=0.2:\n", ECD)

# # Test ansatz_gate and extraction
# V = ansatz_gate(1.0, 0.2, 0.5, 1.0, L, B)
# V_block = extract_ancilla_block(V, L)
# print("Ansatz gate V:\n", V)
# print("Ancilla subspace block V[0:L, 0:L]:\n", V_block)

# Run full optimization (example with callback printing current parameters)
def my_callback(xk):
    print("Current parameters:", xk)

result = optimize_pauli_word("IZZI", lambda0=0.1, beta0=0.1, theta0=0.1, phi0=0.1, callback=my_callback)
print("Optimized parameters:", result.x)
print("Minimum loss:", result.fun)
